<a href="https://colab.research.google.com/github/Malikabriq/smartlearn-fyp/blob/main/my_chatbot_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1 - Install dependencies
!pip install -q --upgrade pip
!pip install -q streamlit pyngrok crawl4ai langchain transformers markdownify nest_asyncio psutil sentence-transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.4 MB/s eta 0:00:00


In [ ]:
# Cell 2 - Write Streamlit app
%%writefile app_local_agent_fixed.py
import streamlit as st
import os, json, asyncio, markdownify, nest_asyncio, traceback, torch
import zipfile, io
from typing import List
from crawl4ai import AsyncWebCrawler
from langchain.schema import Document
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Make async work inside Streamlit/Colab
nest_asyncio.apply()

# ------------------- Paths -------------------
BASE_PATH = "/content"
DOCS_DIR = os.path.join(BASE_PATH, "local_docs")
MD_DIR = os.path.join(BASE_PATH, "scraped_markdown")
DOC_PATH = os.path.join(DOCS_DIR, "stored_docs.json")

os.makedirs(DOCS_DIR, exist_ok=True)
os.makedirs(MD_DIR, exist_ok=True)

# ------------------- Helpers -------------------
async def crawl_site_async(url: str, max_pages: int):
    async with AsyncWebCrawler() as crawler:
        res = await crawler.arun(url=url, max_pages=max_pages, markdown=True)
        return res

def save_as_json(docs: List[Document], append=True):
    all_docs = []
    if append and os.path.exists(DOC_PATH):
        try:
            with open(DOC_PATH, "r", encoding="utf-8") as f:
                all_docs = json.load(f)
        except Exception:
            all_docs = []

    for d in docs:
        all_docs.append({"page_content": d.page_content, "metadata": d.metadata})

    with open(DOC_PATH, "w", encoding="utf-8") as f:
        json.dump(all_docs, f, ensure_ascii=False, indent=2)

    return len(all_docs)

def load_docs():
    if not os.path.exists(DOC_PATH):
        return []
    try:
        with open(DOC_PATH, "r", encoding="utf-8") as f:
            return json.load(f)
    except:
        return []

# ------------------- Streamlit UI -------------------
st.set_page_config(page_title="Web → Local JSON + QA Agent", layout="wide")
st.title("🕸 Web → Local JSON + Local QA Agent")

tab1, tab2 = st.tabs(["🕷 Crawl & Store", "🤖 Ask the Agent"])

# --------- Tab 1: Crawl ----------
with tab1:
    st.header("Crawl a page or site and store as local documents (JSON + Markdown)")
    crawl_mode = st.radio("Extraction Type:", ["Single Page", "Whole Website"])
    url = st.text_input("Enter URL:", value="https://www.bbc.com/")
    save_mode = st.radio("When saving:", ["Overwrite JSON file", "Append to JSON file"])
    do_crawl = st.button("🕷 Crawl & Save")

    if do_crawl:
        if not url.startswith("http"):
            st.warning("Enter a valid URL starting with http/https.")
        else:
            max_pages = 1 if crawl_mode == "Single Page" else 5
            with st.spinner("Crawling website..."):
                try:
                    data = asyncio.run(crawl_site_async(url, max_pages))
                except Exception as e:
                    st.error(f"Error: {e}")
                    st.error(traceback.format_exc())
                    data = None

            if data:
                pages = data.pages if hasattr(data, "pages") else [data]
                docs = []
                for p in pages:
                    md_text = getattr(p, "markdown", None)
                    if not md_text:
                        md_text = markdownify.markdownify(getattr(p, "content", "") or "")

                    # Clean
                    clean = []
                    for line in md_text.splitlines():
                        line = line.strip()
                        if not line: continue
                        if any(x in line.lower() for x in ["copyright", "cookie", "advertisement", "accept"]):
                            continue
                        clean.append(line)
                    md_text = "\n".join(clean)

                    # Save .md locally
                    fname = os.path.join(MD_DIR, f"page_{len(os.listdir(MD_DIR)) + 1}.md")
                    with open(fname, "w", encoding="utf-8") as f:
                        f.write(md_text)

                    docs.append(Document(page_content=md_text, metadata={"source": p.url if hasattr(p, "url") else url}))

                total = save_as_json(docs, append=(save_mode == "Append to JSON file"))
                st.success(f"✅ Saved {len(docs)} page(s). Total stored documents: {total}")

    st.markdown("---")
    st.subheader("📦 Local Data")

    stored = load_docs()
    st.write(f"🧾 Documents stored: *{len(stored)}*")

    # ✅ Download JSON
    if os.path.exists(DOC_PATH):
        with open(DOC_PATH, "rb") as f:
            st.download_button("⬇ Download stored_docs.json", f, "stored_docs.json", "application/json")

    # ✅ Download ZIP of Markdown files
    if len(os.listdir(MD_DIR)) > 0:
        buffer = io.BytesIO()
        with zipfile.ZipFile(buffer, "w", zipfile.ZIP_DEFLATED) as z:
            for filename in os.listdir(MD_DIR):
                z.write(os.path.join(MD_DIR, filename), filename)
        buffer.seek(0)
        st.download_button("⬇ Download Markdown ZIP", buffer, "scraped_markdown.zip", "application/zip")

    # Preview
    if stored:
        preview = st.slider("Preview how many docs:", 1, min(10, len(stored)), 3)
        for i, d in enumerate(stored[:preview]):
            st.markdown(f"### Doc {i+1} — Source: {d['metadata'].get('source','-')}")
            st.code(d["page_content"][:500] + "...", language="markdown")

# --------- Tab 2: QA Agent ----------
with tab2:
    st.header("Ask the local QA agent (NO internet, answers from stored documents only)")
    docs = load_docs()
    query = st.text_input("Enter your question:")
    ask = st.button("🔍 Ask")

    if ask:
        if not docs:
            st.warning("No documents available. Crawl first.")
        else:
            with st.spinner("Generating answer..."):
                context = "\n\n".join(d["page_content"] for d in docs)[:7000]
                model_name = "google/flan-t5-small"
                tokenizer = AutoTokenizer.from_pretrained(model_name)
                model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
                qa = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

                prompt = f"Use only the context below.\n\nContext:\n{context}\n\nQuestion: {query}\n\nIf not in context reply: ❌ Not found."
                out = qa(prompt)[0]['generated_text']
                st.success(out)

Overwriting app_local_agent_fixed.py


In [ ]:
# Cell 3 - Launch Streamlit + ngrok (safe launcher)
import os, time, psutil, signal, threading, socket, subprocess
from pyngrok import ngrok

# 1) find the app file we just wrote
APP_FILE = "app_local_agent_fixed.py"
if not os.path.exists(APP_FILE):
    raise FileNotFoundError(f"App file not found: {APP_FILE}")

# 2) kill previous streamlit/ngrok processes
for p in psutil.process_iter(['pid', 'name']):
    try:
        if p.info['name'] and ('streamlit' in p.info['name'] or 'ngrok' in p.info['name']):
            os.kill(p.info['pid'], signal.SIGTERM)
    except Exception:
        pass

# 3) (optional) set ngrok token
NGROK_TOKEN = "2xRzSISIlaRrLMhqAs5DFLKOqB8_2Y1Q9CiCFYJas4Mes6fnp"  # <-- put your token here if you have one (recommended)
if NGROK_TOKEN:
    ngrok.set_auth_token(NGROK_TOKEN)

env = os.environ.copy()
env["STREAMLIT_SERVER_HEADLESS"] = "true"
cmd = ["streamlit", "run", APP_FILE, "--server.port", "8501", "--server.address", "0.0.0.0"]
proc = subprocess.Popen(cmd, env=env)

def wait_for_port(port, timeout=120):
    start = time.time()
    while time.time() - start < timeout:
        try:
            with socket.create_connection(("127.0.0.1", port), timeout=2):
                return True
        except OSError:
            time.sleep(2)
    return False

print("⏳ Waiting for Streamlit to start (up to 120s)...")
if wait_for_port(8501, timeout=120):
    print("✅ Streamlit started on localhost:8501")
    public_url = ngrok.connect(8501)
    print("🌍 Public app URL:", public_url.public_url)
    print("🔗 (If the page shows ERR_NGROK_8012, wait a few seconds and refresh.)")
else:
    print("❌ Streamlit did not start in time. Check logs with: !streamlit logs")


⏳ Waiting for Streamlit to start (up to 120s)...
✅ Streamlit started on localhost:8501
🌍 Public app URL: https://28ad1100f191.ngrok-free.app
🔗 (If the page shows ERR_NGROK_8012, wait a few seconds and refresh.)
